###  <h1 style="Color:#FF33EC;" >Segmenting and Clustering Neighborhoods in Toronto </h1>


## <p style="color:#239B56;" > Import `BeautifulSoup` Package to scrap the data </p>
<h3>Step 1: Import the requests library and import pandas library</h3>

In [55]:
import requests

In [56]:
from bs4 import BeautifulSoup
import pandas as pd

<h3>Step 2: Send an HTTP request, get the response, and save in a variable</h3>

In [57]:
requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

<Response [200]>

<h3>Step 3: Check the response status code to see if everything went as planned</h3>
<li>status code 200: the request response cycle was successful
<li>any other status code: it didn't work (e.g., 404 = page not found)

In [61]:
#print(response.status_code)

In [46]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

### Retrieve data from wikipedia  page content  

In [26]:
soup = BeautifulSoup(response, 'html.parser')

data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

### Data cleaning

In [51]:
df=pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood'])

In [52]:
# Find indexes of rows that have "Not assigned" in Borough column
indexNames = df[(df['Borough'] == "Not assigned")].index

# Drop rows that have "Not assigned" in Borough column
df.drop(indexNames,inplace=True)

# Drop the first row
df.dropna(inplace=True)

In [53]:
df.head(10)

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Malvern / Rouge
12,M3B,North York,Don Mills
13,M4B,East York,Parkview Hill / Woodbine Gardens
14,M5B,Downtown Toronto,"Garden District, Ryerson"


Combine multiple rows into one row based on Postal code and Borough

In [63]:

df=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

## clean data 

In [70]:
# Replace "Not assigned" in Neighborhood column with the value in Borough column
def custom_fx(data):
    if data['Neighborhood']=='Not assigned':
        var=data['Borough']
    else:
        var=data['Neighborhood']
    return var

# Apply the function
df['Neighborhood']=df.apply(custom_fx,axis='columns')

# Check that there is no more "Not assigned" in Neighborhood column
print("There are {} rows that have 'Not assigned' in Neighborhood column in the dataframe".format(
    len(df[df['Neighborhood']=='Not assigned'])
)
     )
df.head(11)

There are 0 rows that have 'Not assigned' in Neighborhood column in the dataframe


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [66]:
# Export the dataframe
df.to_csv(r"C:\Users\VISH\Desktop\Book1.csv",index=None,header=True)

### Explore the data

### Shape of the table

In [75]:
df.shape

(103, 3)